# Agent Gateway Quickstart

Agent Gateway is a multi-agent framework that offers native support for Snowflake tools. 

The system can be configured to work with 3 types of tools:
- Cortex Search Tool: For unstructured data analysis, which requires a standard RAG access pattern.
- Cortex Analyst Tool: For supporting structured data analysis, which requires a Text2SQL access pattern.
- Python Tool: For supporting custom user operations (using 3rd Party API's), which requires calling arbitrary python.

This notebook walks through how to configure and run a system with all 3 types of tools. The demo is designed to illustrate how the agent can answer questions that require a divserse combination of tools (RAG,Text2SQL, Python, or a combination).

Note that Agent Gateway does not configure the underlying Cortex Search or Cortex Analyst services for the user. Those services must be configured before initializing the agent.

# Agent Configuration

## Connection Setup

Authenticate with Snowpark + set token as environment variable for use by the agents.

In [2]:
from agent_gateway import Agent
from agent_gateway.tools import CortexSearchTool, CortexAnalystTool, PythonTool
from snowflake.snowpark import Session
import os

connection_parameters = {
    "account": os.getenv("SNOWFLAKE_ACCOUNT"),
    "user": os.getenv("SNOWFLAKE_USER"),
    "password": os.getenv("SNOWFLAKE_PASSWORD"),
    "role": os.getenv("SNOWFLAKE_ROLE"),
    "warehouse": os.getenv("SNOWFLAKE_WAREHOUSE"),
    "database": os.getenv("SNOWFLAKE_DATABASE"),
    "schema": os.getenv("SNOWFLAKE_SCHEMA"),
}

snowpark = Session.builder.configs(connection_parameters).create()

## Snowflake Tool Configuration

The Cortex Search Tool and the Cortex Analyst Tool need to be configured as follows. Note that a connection object is required for each config. In the case below we're using the same connection object for both because the services are both in the same account/database/schema. Users have the option to pass in different connection objects as needed.

In [3]:
search_config = {
    "service_name": "SEC_SEARCH_SERVICE",
    "service_topic": "Snowflake's business,product offerings,and performance.",
    "data_description": "Snowflake annual reports",
    "retrieval_columns": ["CHUNK"],
    "snowflake_connection": snowpark,
    "k": 10,
}

analyst_config = {
    "semantic_model": "sp500_semantic_model.yaml",
    "stage": "ANALYST",
    "service_topic": "S&P500 company and stock metrics",
    "data_description": "a table with stock and financial metrics about S&P500 companies ",
    "snowflake_connection": snowpark,
}

## Python Tool Configuration

Configuring a Python Tool for the Agent Gateway requires 1) Python Callable 2) Tool Description (what does the tool do) 3) Output Description (what does the tool output). 

In the example below we create a NewsTool object that submits a HTTP request to a 3rd Party News API. The python callable is passed into the Python Tool as `news_api_func`.To use the tool below get your free token by signing up for an account at thenewsapi.com or just create your own python function and pass it into the PythonTool.

In [4]:
import requests


def html_crawl(url):
    response = requests.get(url)
    return response.text


python_crawler_config = {
    "tool_description": "reads the html from a given URL or website",
    "output_description": "html of a webpage",
    "python_func": html_crawl,
}

## Agent Config

After the tools have been configured, initialize them and configure the agent.

In [5]:
annual_reports = CortexSearchTool(**search_config)
sp500 = CortexAnalystTool(**analyst_config)
web_crawler = PythonTool(**python_crawler_config)

snowflake_tools = [annual_reports, sp500, web_crawler]
agent = Agent(snowflake_connection=snowpark, tools=snowflake_tools, max_retries=3)

2025-03-12 18:53:47,734 - AgentGatewayLogger - INFO - Cortex Search Tool successfully initialized
2025-03-12 18:53:47,735 - AgentGatewayLogger - INFO - Cortex Analyst Tool successfully initialized
2025-03-12 18:53:47,736 - AgentGatewayLogger - INFO - Python Tool successfully initialized
2025-03-12 18:53:47,737 - AgentGatewayLogger - INFO - Cortex gateway successfully initialized


# Agent Testing

The 3 types of questions below are designed to showcase the breadth of tool use patterns possible with the Agent Gateway. 

- The Structured Data Questions use the Cortex Analyst agent. 
- The Unstructured Data Questions use either the Cortex Search agent or the Python (News API) agent.
- The last section includes a question that requires the use of both types of tools.

## Structured Data Questions

In [6]:
agent("What is the market cap of Apple?")

2025-03-12 18:53:51,997 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task


'The market cap of Apple is $3,019,131,060,224.'

In [7]:
agent("Which company has the bigger EBITDA, Apple or MSFT?")

2025-03-12 18:54:02,991 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task
2025-03-12 18:54:02,994 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task


'Apple has a bigger EBITDA than Microsoft.'

# Unstructured Data Questions

In [8]:
agent("How many customers does Snowflake have?")

2025-03-12 18:54:13,290 - AgentGatewayLogger - INFO - running sec_search_service_cortexsearch task


'As of January 31, 2024, Snowflake had 9,437 total customers.'

In [9]:
agent(
    "On which platforms can I host Snowflake according to this documentation page https://docs.snowflake.com/en/user-guide/intro-cloud-platforms"
)

2025-03-12 18:54:19,584 - AgentGatewayLogger - INFO - running html_crawl task


'Snowflake can be hosted on Amazon Web Services (AWS), Google Cloud Platform (GCP), and Microsoft Azure (Azure).'

## Unstructured + Structured Data Questions

In [10]:
agent(
    "What is the market cap of the cloud providers mentioned in Snowflake's annual report?"
)

2025-03-12 18:54:36,542 - AgentGatewayLogger - INFO - running sec_search_service_cortexsearch task
2025-03-12 18:54:37,251 - AgentGatewayLogger - INFO - running summarize task
2025-03-12 18:54:43,381 - AgentGatewayLogger - INFO - running sp500_semantic_model_cortexanalyst task


"The market cap of the cloud providers mentioned in Snowflake's annual report are: Microsoft Corporation (Azure) - $3,150,184,448,000, Alphabet Inc. (Google Cloud Platform) - $2,164,350,779,392, and Amazon.com, Inc. (AWS) - $1,917,936,336,896."